In [ ]:
import mysql.connector
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
import pandas as pd

# Connect to MySQL
def connect_db():
    try:
        global myconn
        myconn = mysql.connector.connect(host="localhost", user="root", passwd="Welcome@2020")
        messagebox.showinfo("Success", "Connected to MySQL Server")
    except mysql.connector.Error as err:
        messagebox.showerror("Error", f"Connection Failed: {err}")

# Fetch and display databases
def show_databases():
    try:
        cur = myconn.cursor()
        cur.execute("SHOW DATABASES")
        databases_listbox.delete(0, tk.END)
        for db in cur:
            databases_listbox.insert(tk.END, db[0])
    except mysql.connector.Error as err:
        messagebox.showerror("Error", f"Failed to fetch databases: {err}")

# Fetch and display tables
def show_tables():
    try:
        selected_db = databases_listbox.get(tk.ACTIVE)
        if not selected_db:
            messagebox.showwarning("Warning", "Select a database first")
            return

        global selected_database
        selected_database = selected_db
        myconn.database = selected_db

        cur = myconn.cursor()
        cur.execute("SHOW TABLES")
        tables_listbox.delete(0, tk.END)
        for table in cur:
            tables_listbox.insert(tk.END, table[0])
    except mysql.connector.Error as err:
        messagebox.showerror("Error", f"Failed to fetch tables: {err}")

# Show table structure
def show_table_structure():
    try:
        selected_table = tables_listbox.get(tk.ACTIVE)
        if not selected_table:
            messagebox.showwarning("Warning", "Select a table first")
            return

        cur = myconn.cursor()
        cur.execute(f"DESCRIBE {selected_table}")

        structure_text.delete("1.0", tk.END)
        structure_text.insert(tk.END, "Column Name\tData Type\tConstraints\n")
        structure_text.insert(tk.END, "-"*50 + "\n")

        for row in cur:
            structure_text.insert(tk.END, f"{row[0]}\t{row[1]}\t{row[2:]}\n")
    except mysql.connector.Error as err:
        messagebox.showerror("Error", f"Failed to fetch table structure: {err}")

# Fetch and display table data
def show_table_data():
    try:
        selected_table = tables_listbox.get(tk.ACTIVE)
        if not selected_table:
            messagebox.showwarning("Warning", "Select a table first")
            return

        cur = myconn.cursor()
        cur.execute(f"SELECT * FROM {selected_table} LIMIT 10")

        for row in tree.get_children():
            tree.delete(row)

        columns = [desc[0] for desc in cur.description]
        tree["columns"] = columns
        tree["show"] = "headings"

        for col in columns:
            tree.heading(col, text=col)
            tree.column(col, width=120, anchor="center")

        for row in cur:
            tree.insert("", "end", values=row)
    except mysql.connector.Error as err:
        messagebox.showerror("Error", f"Failed to fetch data: {err}")

# GUI Setup
root = tk.Tk()
root.title("SQL Workbench")
root.geometry("850x600")  # Adjusted window size
root.configure(bg="#3B6790")  # Main Background Color

# Button Styling
button_style = {"font": ("Arial", 10, "bold"), "bg": "#EFB036", "fg": "white", "width": 20}

# Labels
tk.Label(root, text="SQL Workbench", bg="#3B6790", fg="white", font=("Arial", 16, "bold")).pack(pady=10)

tk.Button(root, text="Connect to MySQL", command=connect_db, **button_style).pack(pady=5)

# Frame for Databases and Tables
frame = tk.Frame(root, bg="#3B6790")
frame.pack(pady=5)

# Database Section
db_frame = tk.Frame(frame, bg="#3B6790")
db_frame.pack(side=tk.LEFT, padx=10)

tk.Button(db_frame, text="Show Databases", command=show_databases, **button_style).pack()
databases_listbox = tk.Listbox(db_frame, height=10, width=40, bg="#23486A", fg="white")
databases_listbox.pack(pady=5)

# Tables Section
table_frame = tk.Frame(frame, bg="#3B6790")
table_frame.pack(side=tk.LEFT, padx=10)

tk.Button(table_frame, text="Show Tables", command=show_tables, **button_style).pack()
tables_listbox = tk.Listbox(table_frame, height=10, width=40, bg="#23486A", fg="white")
tables_listbox.pack(pady=5)

# Table Structure Section
tk.Button(root, text="Show Table Structure", command=show_table_structure, **button_style).pack(pady=5)
structure_text = tk.Text(root, height=6, width=85, bg="#23486A", fg="white")
structure_text.pack(pady=5)

# Table Data Section
tk.Button(root, text="Show Table Data", command=show_table_data, **button_style).pack(pady=5)
tree_frame = tk.Frame(root, bg="#3B6790")
tree_frame.pack(pady=5)

tree = ttk.Treeview(tree_frame, height=8, style="Custom.Treeview")
tree.pack()

style = ttk.Style()
style.configure("Custom.Treeview", background="#23486A", foreground="white", fieldbackground="#23486A")

root.mainloop()